In [3]:
import os
USER = !whoami
USER = USER[0]
OUTPUT_PATH_BASE = '/user/{USER}'.format(USER=USER)

# Using python versions on the cluster nodes
https://documentation.altiscale.com/python-with-hadoop-and-spark

<pre>
The concept of SCL is that a pre-deployed environment is already available. To enable it, you need to tell your 'job' and 'python script' the following two important things.

Location of the Python interpreter on the Workbench and the remote nodes:

Python 2.7: /opt/rh/python27/root/usr/bin/python2.7
Python 3.3: /opt/rh/python33/root/usr/bin/python3.3
Python 3.4: /opt/rh/rh-python34/root/usr/bin/python3.4

LD_LIBRARY_PATH required by the Python interpreter depending on which one of the above versions you are invoking:

Python 2.7: /opt/rh/python27/root/usr/lib64
Python 3.3: /opt/rh/python33/root/usr/lib64
Python 3.4: /opt/rh/rh-python34/root/usr/lib64
</pre>

### Local python version

In [4]:
import platform
platform.python_version()

'2.7.14'

### Let's check what's running on the clusters
Note that even though the shebang points to python 2.7, this is not enough to invoke the 2.7 environment

In [5]:
%%writefile one_line.txt
hello

Overwriting one_line.txt


In [22]:
%%writefile mrjob_versions_test.py
#!/usr/bin/env /opt/rh/python27/root/usr/bin/python2.7
from mrjob.compat import jobconf_from_env
from mrjob.job import MRJob
from mrjob.step import MRStep

import platform
class MRCountLinesByFile(MRJob):

    def mapper(self, _, line):
        yield None, str(platform.python_version())


        
    def steps(self):
        JOBCONF_STEP = {        
           'mapreduce.job.reduces': '0' 
        }
        return [MRStep(
            jobconf=JOBCONF_STEP,    
            mapper = self.mapper
        )]

if __name__ == '__main__':
    MRCountLinesByFile.run()

Overwriting mrjob_versions_test.py


In [16]:
OUTPUT_PATH_0 = os.path.join(OUTPUT_PATH_BASE,'tests0')
!hadoop fs -rm -r {OUTPUT_PATH_0}
!python mrjob_versions_test.py \
        -r hadoop one_line.txt \
        --output-dir={OUTPUT_PATH_0} \
        --no-output

17/10/10 20:39:33 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/kylehamilton/tests0' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/kylehamilton/.Trash/Current
No configs found; falling back on auto-configuration
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/mrjob_versions_test.kylehamilton.20171010.203934.680283
Copying local files to hdfs:///user/kylehamilton/tmp/mrjob/mrjob_versions_test.kylehamilton.20171010.203934.680283/files/...
Running step 1 of 1...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob7798724538349123326.jar tmpDir=null
  Timeline service a

In [18]:
!hadoop fs -cat {OUTPUT_PATH_0}/*

null	"2.6.6"


### Invoke python 2.7

In [19]:
OUTPUT_PATH_1 = os.path.join(OUTPUT_PATH_BASE,'tests1')
!hadoop fs -rm -r {OUTPUT_PATH_1}
!python mrjob_versions_test.py \
        -r hadoop one_line.txt \
        --python-bin /opt/rh/python27/root/usr/bin/python2.7 \
        --cmdenv LD_LIBRARY_PATH=/opt/rh/python27/root/usr/lib64 \
        --output-dir={OUTPUT_PATH_1} \
        --no-output
        

17/10/10 20:41:28 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 5760 minutes, Emptier interval = 360 minutes.
Moved: 'hdfs://nn-ia.s3s.altiscale.com:8020/user/kylehamilton/tests1' to trash at: hdfs://nn-ia.s3s.altiscale.com:8020/user/kylehamilton/.Trash/Current
No configs found; falling back on auto-configuration
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/mrjob_versions_test.kylehamilton.20171010.204129.278307
Copying local files to hdfs:///user/kylehamilton/tmp/mrjob/mrjob_versions_test.kylehamilton.20171010.204129.278307/files/...
Running step 1 of 1...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob8720540373873707317.jar tmpDir=null
  Timeline service a

In [21]:
!hadoop fs -cat {OUTPUT_PATH_1}/*

null	"2.7.8"


### Invoke python 3
Remember to update the shebang in your script!!
```
#!/usr/bin/env /opt/rh/python33/root/usr/bin/python3
```

In [23]:
OUTPUT_PATH_2 = os.path.join(OUTPUT_PATH_BASE,'tests2')
!hadoop fs -rm -r {OUTPUT_PATH_2}
!python mrjob_versions_test.py \
        -r hadoop one_line.txt \
        --python-bin /opt/rh/python33/root/usr/bin/python3 \
        --cmdenv LD_LIBRARY_PATH=/opt/rh/python33/root/usr/lib64 \
        --output-dir={OUTPUT_PATH_2} \
        --no-output

rm: `/user/kylehamilton/tests2': No such file or directory
No configs found; falling back on auto-configuration
Looking for hadoop binary in /opt/hadoop/bin...
Found hadoop binary: /opt/hadoop/bin/hadoop
Using Hadoop version 2.7.3
Looking for Hadoop streaming jar in /opt/hadoop...
Found Hadoop streaming jar: /opt/hadoop/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar
Creating temp directory /tmp/mrjob_versions_test.kylehamilton.20171010.212851.930854
Copying local files to hdfs:///user/kylehamilton/tmp/mrjob/mrjob_versions_test.kylehamilton.20171010.212851.930854/files/...
Running step 1 of 1...
  packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob7189154233950240525.jar tmpDir=null
  Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v1/timeline/
  Connecting to ResourceManager at rm-ia.s3s.altiscale.com/10.251.255.108:8032
  Connecting to Application History server at rm-ia.s3s.altiscale.com/10.251.255.108:10200
  Tim

In [25]:
!hadoop fs -cat {OUTPUT_PATH_2}/*

null	"3.3.2"


# Hadoop streaming example

In [2]:
%%writefile test_py3.py
#!/usr/bin/env /opt/rh/python33/root/usr/bin/python3

import os
import time
import subprocess
import platform
from mrjob.job import MRJob


if __name__ == "__main__":
    
    print(str(platform.python_version()))
    
    python_default_runtime_path = subprocess.Popen(["which", "python"], stdout=subprocess.PIPE)
    result_lines = python_default_runtime_path.communicate()[0]
    lines = result_lines.decode("utf-8")
    print(lines)

    list_runtime_files_result = subprocess.Popen(["ls", "-alR"], stdout=subprocess.PIPE)
    result_lines = list_runtime_files_result.communicate()[0]
    lines = result_lines.decode("utf-8")
    print(lines)

    env_proc_result = subprocess.Popen(["env"], stdout=subprocess.PIPE)
    result_lines = env_proc_result.communicate()[0]
    lines = result_lines.decode("utf-8")
    print(lines)

    mount = subprocess.check_output(["mount" ,"-v"])
    lines = mount.decode("utf-8").splitlines()
    mount_lists = map(lambda line: line.split()[2], lines)
    for fpath in mount_lists:
        print(fpath)

    print(os.environ)

Overwriting test_py3.py


In [3]:
%%bash
#!/bin/bash

HADOOP_VERSION=2.7.1
TEST_PID=$$
HDFS_TEST_OUTPUT=/tmp/scl-kyle-test_${TEST_PID}

# Generate a random sample and upload to HDFS
# Default goes to /user/$USER/randomsample.txt
echo -e "1\n2\n3\n4\n5\n6\n7\n8\n9" > "randomsample.txt"

# Ignore HDFS errors here
hdfs dfs -put randomsample.txt
# Ignore error from HDFS destination existance, can ignore
hdfs dfs -rm -r $HDFS_TEST_OUTPUT

# Launch Hadoop Streaming with SCL configured properly
# Reducer # is set to 0 on purpose to avoid shuffle + sort on results
hadoop jar /opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar -files test_py3.py -mapper test_py3.py -numReduceTasks 0 -input /user/$USER/randomsample.txt -output $HDFS_TEST_OUTPUT -cmdenv LD_LIBRARY_PATH=/opt/rh/python33/root/usr/lib64

packageJobJar: [] [/opt/hadoop-2.7.3/share/hadoop/tools/lib/hadoop-streaming-2.7.3.jar] /tmp/streamjob8786421729991228186.jar tmpDir=null


put: `randomsample.txt': File exists
rm: `/tmp/scl-kyle-test_75516': No such file or directory
17/10/10 11:35:33 INFO impl.TimelineClientImpl: Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v1/timeline/
17/10/10 11:35:33 INFO client.RMProxy: Connecting to ResourceManager at rm-ia.s3s.altiscale.com/10.251.255.108:8032
17/10/10 11:35:33 INFO client.AHSProxy: Connecting to Application History server at rm-ia.s3s.altiscale.com/10.251.255.108:10200
17/10/10 11:35:33 INFO impl.TimelineClientImpl: Timeline service address: http://rm-ia.s3s.altiscale.com:8188/ws/v1/timeline/
17/10/10 11:35:33 INFO client.RMProxy: Connecting to ResourceManager at rm-ia.s3s.altiscale.com/10.251.255.108:8032
17/10/10 11:35:33 INFO client.AHSProxy: Connecting to Application History server at rm-ia.s3s.altiscale.com/10.251.255.108:10200
17/10/10 11:35:35 INFO lzo.GPLNativeCodeLoader: Loaded native gpl library from the embedded binaries
17/10/10 11:35:35 INFO lzo.LzoCodec: Successfully loaded & ini

In [4]:
!hdfs dfs -cat /tmp/scl-kyle-test_36925/part-00000

3.3.2	
/usr/bin/python	
	
.:	
total 164	
drwxr-s--- 3 kylehamilton yarn   4096 Oct  8 14:49 .	
drwxr-s--- 7 kylehamilton yarn   4096 Oct  8 14:49 ..	
-rw-r----- 1 kylehamilton yarn   1012 Oct  8 14:49 .job.xml.crc	
-rw------- 1 kylehamilton yarn    129 Oct  8 14:49 container_tokens	
lrwxrwxrwx 1 kylehamilton yarn    113 Oct  8 14:49 job.jar -> /var/storage/nm-sdi1/nm-local/usercache/kylehamilton/appcache/application_1506640654827_1077/filecache/11/job.jar	
-rw-r----- 1 kylehamilton yarn 128495 Oct  8 14:49 job.xml	
-rwx------ 1 kylehamilton yarn   7091 Oct  8 14:49 launch_container.sh	
lrwxrwxrwx 1 kylehamilton yarn     77 Oct  8 14:49 test_py3.py -> /var/storage/nm-sdg1/nm-local/usercache/kylehamilton/filecache/13/test_py3.py	
drwxr-s--- 2 kylehamilton yarn   4096 Oct  8 14:49 tmp	
	
./tmp:	
total 80	
drwxr-s--- 2 kylehamilton yarn  4096 Oct  8 14:49 .	
drwxr-s--- 3 kylehamilton yarn  4096 Oct  8 14:49 ..	
-rwxr-x--x 1 kylehamilton yarn 69643 Oct  8 14:49 unpacked-8941739013605427874-